In [121]:
import pandas as pd
import datetime
from gsheets_connect import GoogleSheets, Connect
import helper_functions as hlp
from expense import Expense

In [104]:
mnths_dict = {
    'Ene':'01','Feb':'02','Mar':'03',
    'Abr':'04','May':'05','Jun':'06',
    'Jul':'07','Ago':'08','Sep':'09',
    'Oct':'10','Nov':'11','Dic':'12'
}

In [113]:
data = pd.read_html('5470467005458188 - 17_Ene_20.xls')

cols = data[1].loc[5,:].drop(1)
cols.name = 'ix'

data = data[1].loc[6:,:].drop(1, axis=1)

data.columns = cols
data.reset_index(inplace=True, drop=True)

In [114]:
from datetime import datetime

In [115]:
d = data.loc[0, 'FECHA']

In [116]:
def parse_month(date_string):
    for m,v in mnths_dict.items():
        if m in date_string:
            date_string = date_string.replace(m,v)
    return date_string

In [117]:
data.iloc[:,0] = data.iloc[:,0].apply(
    lambda d: parse_month(d)
).apply(
    lambda d: datetime.strptime(d, '%d/%m/%Y')
)

data.iloc[:,2] = data.iloc[:,2].astype(float)

In [120]:
positive_data = data.loc[data.iloc[:,2]>0]

In [122]:
cols = [
    "Amount $",
    "Expense Day", 
    "Expense Month", 
    "Payment Day", 
    "Payment Month", 
    "Installments", 
    "Payment Method",
    "Method Name",
    "Category", 
    "Sub Category",
    "Payment Fortnight"
]
new_expenses_df = pd.DataFrame(columns=cols)

In [123]:
new_expenses_df

,Amount $,Expense Day,Expense Month,Payment Day,Payment Month,Installments,Payment Method,Method Name,Category,Sub Category,Payment Fortnight


In [ ]:
new_expenses = []
for ix, row in positive_data.iterrows():
    print('')
    print('----------------------------------')
    print("Currently Processing this expense:")
    print(row)
    #new_expense = Expense()
    inpt = input('Continue? ')
    if inpt != 'y':
        break


----------------------------------
Currently Processing this expense:
ix
FECHA          2020-01-15 00:00:00
CONCEPTO    S WORLD PATRIOTISMO VD
IMPORTE                      19430
Name: 0, dtype: object
Continue? y

----------------------------------
Currently Processing this expense:
ix
FECHA          2020-01-15 00:00:00
CONCEPTO    S WORLD PATRIOTISMO VD
IMPORTE                       2459
Name: 1, dtype: object


In [2]:
GSHEET_ID = '14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4'

In [3]:
gsheet = GoogleSheets()

In [5]:
data = hlp.retrieveDataFromSheet(
    gsheet, 
    GSHEET_ID, 
    sheet_name = 'automatic spends loader'
)

In [9]:
import arrow

In [8]:
data['escrom'] = data['Date'].apply(lambda x: arrow.get)

0    15/01/2020
1    16/01/2020
2    17/01/2020
3    17/01/2020
Name: Date, dtype: object

In [5]:
cur_data = pd.read_csv(r'daily_data.txt', sep=' ')

#cur_data['Payment Day'] = pd.to_datetime(cur_data['Payment Day']).dt.date

cur_data['Payment Fortnight'] = cur_data['Payment Day'].apply(
    lambda date: str(pd.to_datetime(date).replace(day=15 if pd.to_datetime(date).day <= 15 else 30).date())
)

In [6]:
hlp.dataFrameToListOfValues(cur_data)

array([['Amount $', 'Expense Day', 'Expense Month', 'Payment Day',
        'Payment Month', 'Installments', 'Payment Method', 'Method Name',
        'Category', 'Sub Category', 'Payment Fortnight'],
       [267.0, '2019-08-01', 201908.0, '2019-08-01', 201908.0, 1.0,
        'debit', 'debit', 'Food', 'Restaurants', '2019-08-15'],
       [633.0, '2019-08-12', 201908.0, '2019-08-15', 201908.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-08-15'],
       [633.0, '2019-08-12', 201908.0, '2019-09-15', 201909.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-09-15'],
       [633.0, '2019-08-12', 201908.0, '2019-10-15', 201910.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-10-15'],
       [633.0, '2019-08-12', 201908.0, '2019-11-15', 201911.0, 1.0,
        'credit', 'Santander-Free', 'Personal Stuff', 'Stuff for me',
        '2019-11-15'],
       [633.0, '2019-08-1

In [7]:
gsheet.values_to_gsheet(
    gsheet_id = GSHEET_ID,
    values_list = hlp.dataFrameToListOfValues(cur_data).tolist(),  
    range_start='data'
)

220 cells updated.


In [5]:
cols = np.array(cur_data.columns)

cols_values = cols.tolist()

values_arr = cur_data.values

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [9]:
request = gsheet.service.spreadsheets().values().clear(
    spreadsheetId=GSHEET_ID, 
    range='bimonthly data' 
    #body=clear_values_request_body
)
response = request.execute()

In [6]:
categories_titles = gsheet.gsheet_to_df(
    spreadsheet_id = '14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4',
    range_name = 'biweekly plan!B27:D100'
)

In [92]:
cur_data = gsheet.gsheet_to_df(
    spreadsheet_id = '14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4',
    range_name = 'data'
)

In [95]:
cur_data.columns

Index(['Amount $', 'Expense Day', 'Expense Month', 'Payment Day',
       'Payment Month', 'Installments', 'Payment Method', 'Method Name',
       'Category', 'Sub Category', 'Payment Fortnight'],
      dtype='object')

In [7]:
separations = categories_titles[
    categories_titles['Variable Expenses'].isnull()
].index

In [8]:
separations#[:-1]

Int64Index([0, 7, 12, 18, 24, 32, 37], dtype='int64')

In [15]:
cats = {}
sub_cats = {}
for i in range(1,len(separations)):
    ix_1 = separations[i-1]+1
    ix_2 = separations[i]-1
    cat = categories_titles.loc[ix_1:ix_2, 'Variable Expenses']
    cats[i] =  cat.iloc[0]
    sc = {}
    for j, c in enumerate(cat.iloc[1:]):
        sc[j+1] = c
    sub_cats[cat.iloc[0]] = sc
cat = categories_titles.loc[separations[-1]+1:, 'Variable Expenses']
cats[len(separations)] =  cat.iloc[0]
sc = {}
for j, c in enumerate(cat.iloc[1:]):
    sc[j+1] = c
sub_cats[cat.iloc[0]] = sc

In [25]:
cats_template = hlp.categoriesTemplate(sub_cats).sort_index()
cats_template['range'] = np.arange(1, cats_template.shape[0]+1)

In [33]:
pivot = pd.pivot_table(
    cur_data, 
    values = 'Amount $',
    columns='Payment Fortnight',
    aggfunc='sum',
    index=['Category', 'Sub Category']
).reset_index()

In [75]:
data_with_all_cats = pd.merge(
    cats_template, 
    pivot, 
    on = ['Category', 'Sub Category'], 
    how='left'
).fillna(0)

pivoted_to_write_on_gsheet = pd.DataFrame(
    data_with_all_cats.set_index(
        ['Category', 'Sub Category', 'range']
    ).stack()
).reset_index().rename(
    columns={
        'level_3': 'Payment Fortnight', 
        0: 'Amount $'
    }
)

In [77]:
pivoted_to_write_on_gsheet = pivoted_to_write_on_gsheet.loc[
    :,
    ['range', 'Category', 'Sub Category', 'Payment Fortnight', 'Amount $']
]

In [78]:
cols = np.array(pivoted_to_write_on_gsheet.columns)

cols_values = cols.tolist()

values_arr = pivoted_to_write_on_gsheet.values#.tolist()

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [80]:
data = [
    {
        'range': 'biweekly data!B2',
        'values': list_of_rows.tolist()
    }
]
body = {
    'valueInputOption': 'RAW',
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId='14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4', body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))

1625 cells updated.


In [81]:
cur_data.columns

Index(['Amount $', 'Expense Day', 'Expense Month', 'Payment Day',
       'Payment Month', 'Installments', 'Payment Method', 'Method Name',
       'Category', 'Sub Category', 'Payment Fortnight'],
      dtype='object')

In [84]:
monthly_pivot = pd.pivot_table(
    cur_data, 
    values = 'Amount $',
    columns='Payment Month',
    aggfunc='sum',
    index=['Category', 'Sub Category']
).reset_index()

In [87]:
data_with_all_cats = pd.merge(
    cats_template, 
    monthly_pivot, 
    on = ['Category', 'Sub Category'], 
    how='left'
).fillna(0)

pivoted_to_write_on_gsheet = pd.DataFrame(
    data_with_all_cats.set_index(
        ['Category', 'Sub Category', 'range']
    ).stack()
).reset_index().rename(
    columns={
        'level_3': 'Payment Month', 
        0: 'Amount $'
    }
)

In [88]:
pivoted_to_write_on_gsheet

,Category,Sub Category,range,Payment Month,Amount $
0,Food,Super Market,1,201908.0,0.0
1,Food,Super Market,1,201909.0,0.0
2,Food,Super Market,1,201910.0,0.0
3,Food,Super Market,1,201911.0,0.0
4,Food,Super Market,1,201912.0,0.0
5,Food,Super Market,1,202001.0,0.0
6,Food,Restaurants,2,201908.0,267.0
7,Food,Restaurants,2,201909.0,0.0
8,Food,Restaurants,2,201910.0,0.0
9,Food,Restaurants,2,201911.0,0.0


In [89]:
pivoted_to_write_on_gsheet = pivoted_to_write_on_gsheet.loc[
    :,
    ['range', 'Category', 'Sub Category', 'Payment Month', 'Amount $']
]

In [90]:
cols = np.array(pivoted_to_write_on_gsheet.columns)

cols_values = cols.tolist()

values_arr = pivoted_to_write_on_gsheet.values#.tolist()

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [91]:
data = [
    {
        'range': 'monthly data!B2',
        'values': list_of_rows.tolist()
    }
]
body = {
    'valueInputOption': 'RAW',
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId='14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4', body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))

815 cells updated.


In [13]:
monthly_data#.unstack(0)

Amount $
Payment Fortnight Category       Sub Category            
2019-08-15        Food           Restaurants   267.000000
                  Personal Stuff Stuff for me  633.000000
2019-08-30        Personal Stuff Stuff for me  625.166667
2019-09-15        Personal Stuff Stuff for me  633.000000
2019-09-30        Personal Stuff Stuff for me  625.166667
2019-10-15        Personal Stuff Stuff for me  633.000000
2019-10-30        Personal Stuff Stuff for me  625.166667
2019-11-15        Personal Stuff Stuff for me  633.000000
2019-11-30        Personal Stuff Stuff for me  625.166667
2019-12-15        Personal Stuff Stuff for me  633.000000
2019-12-30        Personal Stuff Stuff for me  625.166667
2020-01-15        Personal Stuff Stuff for me  633.000000
2020-01-30        Personal Stuff Stuff for me  625.166667

In [55]:
monthly_data.unstack(0).shape

(2, 12)

In [58]:
numberToLetters(12)

'L'

In [ ]:
rows_range = 

In [39]:
dates = np.array(monthly_data.unstack(0).columns.levels[1])

cols_values = np.array(['category', 'sub_category']+dates.astype(str).tolist())

values_arr = monthly_data.unstack(0).fillna(0).reset_index().values#.tolist()

list_of_rows = np.insert(values_arr, 0, cols_values, 0)

In [59]:
list_of_rows.shape

(3, 14)

In [71]:
start_from = 'biweekly pivot!B2'

In [66]:
table_range

'B2:N3'

In [72]:
data = [
    {
        'range': start_from,
        'values': list_of_rows.tolist()
    }
]
body = {
    'valueInputOption': 'RAW',
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId='14a36tesQZ2AH0aIEdG5Ch2F8iVyrnXCvlxeASLX47N4', body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))

42 cells updated.


In [9]:
list_of_rows

[['Food',
  'Restaurants',
  267.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 ['Personal Stuff',
  'Stuff for me',
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666,
  633.0,
  625.1666666666666]]